In [1]:
%pwd

'c:\\Users\\aarav\\Downloads\\MoviesRecommendationSystemMLops\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\aarav\\Downloads\\MoviesRecommendationSystemMLops'

In [4]:
import pandas as pd
import numpy as np

In [5]:
from dataclasses import dataclass 
from pathlib import Path  

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    movies_URL: str
    credits_URL: str
    movies_data_file: Path
    credits_data_file: Path
    unzip_dir: Path

In [5]:
# Configuration Manager


In [6]:
from src.MoviesRecommendationSystemMLops.constants import *
from src.MoviesRecommendationSystemMLops.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion 

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                movies_URL=config.movies_URL,
                credits_URL=config.credits_URL,
                movies_data_file=config.movies_data_file,
                credits_data_file=config.credits_data_file,
                unzip_dir=config.unzip_dir
          )       

          return data_ingestion_config

In [8]:
#Components

In [13]:
import os 
import urllib.request as request 
import zipfile
from src.MoviesRecommendationSystemMLops import logger 
from src.MoviesRecommendationSystemMLops.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config   

    def download_movies_file(self):
        if not os.path.exists(self.config.movies_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.movies_URL,
                filename= self.config.movies_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.movies_data_file))}")
    
    
    def download_credits_file(self):
        if not os.path.exists(self.config.credits_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.credits_URL,
                filename= self.config.credits_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.movies_data_file))}")




    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.movies_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.credits_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Updating the Pipeline

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_movies_file()
    data_ingestion.download_credits_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e   

[2024-01-18 17:35:15,799: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-18 17:35:15,806: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-18 17:35:15,809: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-18 17:35:15,814: INFO: common: created directory at: artifacts]
[2024-01-18 17:35:15,818: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-18 17:35:17,475: INFO: 2917063386: artifacts/data_ingestion/movies.zip download! with following info: 
Connection: close
Content-Length: 1666990
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d0e59683d2bf36e68673f68db57c1b94ef09cbcc2ab8059f4c759ec01ad4a179"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 92E2:14E994:1981D6:259F92:65A913F9
Accept-Ranges: bytes
Date: Thu